In [22]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

import seaborn as sns
import numpy as np, time, os
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise import evaluate, print_perf
from surprise import accuracy
from sklearn import cross_validation as cv
from nltk.stem.snowball import SnowballStemmer
from pymongo import MongoClient
%matplotlib inline

In [2]:
client = MongoClient()
db = client['test']
#reviews = pd.DataFrame(list(db.review.find()))

In [8]:
reviews

,_id,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,5adf69e696c8f7b6476d511d,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw
1,5adf69e696c8f7b6476d511e,AEx2SYEUJmTxVVB18LlCwA,0,2016-05-28,0,vkVSCC7xljjrAI4UGfnKEQ,5,Super simple place but amazing nonetheless. It...,0,bv2nCi5Qv5vroFiqKGopiw
2,5adf69e696c8f7b6476d511f,s2I_Ni76bjJNK9yG60iD-Q,0,2016-05-28,0,L_9BTb55X0GDtThi6GlZ6w,4,Had their chocolate almond croissant and it wa...,0,bv2nCi5Qv5vroFiqKGopiw
3,5adf69e696c8f7b6476d5120,VR6GpWIda3SfvPC-lg9H3w,0,2016-05-28,0,n6QzIUObkYshz4dz2QRJTw,5,Small unassuming place that changes their menu...,0,bv2nCi5Qv5vroFiqKGopiw
4,5adf69e696c8f7b6476d5121,ACFtxLv8pGrrxMm6EgjreA,0,2016-05-28,0,IXvOzsEMYtiJI0CARmj77Q,4,Love coming here. Yes the place always needs t...,0,bv2nCi5Qv5vroFiqKGopiw
5,5adf69e696c8f7b6476d5122,8QWPlVQ6D-OExqXoaD2Z1g,0,2014-09-24,0,HRPm3vEZ_F-33TYVT7Pebw,5,Cycle Pub Las Vegas was a blast! Got a groupon...,1,_4iMDXbXZ1p1ONG297YEAQ
6,5adf69e696c8f7b6476d5123,gkCorLgPyQLsptTHalL61g,0,2015-10-27,0,8UIishPUD92hXtScSga_gw,4,Always drove past this coffee house and wonder...,1,u0LXt3Uea_GidxRW1xcsfg
7,5adf69e696c8f7b6476d5124,9_CGhHMz8698M9-PkVf0CQ,2,2012-05-11,0,ymAUG8DZfQcFTBSOiaNN4w,4,Who would have guess that you would be able to...,0,u0LXt3Uea_GidxRW1xcsfg
8,5adf69e696c8f7b6476d5125,5r6-G9C4YLbC7Ziz57l3rQ,0,2013-02-09,0,w41ZS9shepfO3uEyhXEWuQ,3,"Not bad!! Love that there is a gluten-free, ve...",1,u0LXt3Uea_GidxRW1xcsfg
9,5adf69e696c8f7b6476d5126,fDF_o2JPU8BR1Gya--jRIA,0,2016-04-06,0,WF_QTN3p-thD74hqpp2j-Q,5,Love this place!\n\nPeggy is great with dogs a...,3,u0LXt3Uea_GidxRW1xcsfg


In [4]:
df = pd.read_csv( './toronto_reviews.csv')
df = df.rename(columns={'id': 'review_id'})

In [5]:
df

,review_id,stars,user_id,business_id,useful,cool,funny
0,---0hl58W-sjVTKi5LghGw,4,Y6qylbHq8QJmaCRSlKdIog,Ae4ABFarGMaI5lk1i98A0w,0,0,0
1,---OVMmOizYt30bx3-D0gA,5,WH0D3U5UAtNy0X5U5ZeJnw,ixGb4NLfgD90wZHlw94rLw,6,5,1
2,--0pfY3vQilgl20btE0fVQ,4,wbYWgTS-vzgfVYV_C1qhig,5BPv_vAnRp5EKdmacqvrLw,5,3,2
3,--0y7xOZPpiGD5d6vgfafg,1,EUdDBB1fHLq86T0aI9VLtg,vh1tPEaPioD78QmoqnWXpw,1,0,1
4,--1sfdWmiYGbTxpGTy-3ZQ,1,hZgREoZPC30EEilayde2Ug,qAg8GkN9rforFWmGxBqVqA,2,0,0
5,--1Uj2FGirXqBmi_oA6-qg,5,-BJIu6TWs26q2yxozPAbEg,yxdmX8dXk5Se9zJ4WAW1pw,3,0,0
6,--1U_KyCEPuYg2a1q03FbQ,5,0tpJmUYvbQSRbNa6DI0WVA,-yZXurhfOYlNKmH0ZyNGIQ,0,0,0
7,--242HMKec9b90td9y_s5g,5,3fR4DeE_eSa7SbzuC_fsaQ,i1Xkm22_96GNVVOXSUoL0A,0,0,0
8,--2WluEwS6KHmD-2v2V5sw,5,FykmLnIuh6oPaSPJ3RLB_A,aLcFhMe6DDJ430zelCpd2A,0,0,1
9,--3sDRa8FTI_cKLF4eUAeQ,2,dhPp4t9_fD0R0eNsp_gYZw,d6G0w_e3GaA90NSQeovShA,0,0,0


In [6]:

dfReviews = pd.merge(df, reviews, on='review_id' )


In [7]:
dfReviews

,review_id,stars_x,user_id_x,business_id_x,useful_x,cool_x,funny_x,_id,business_id_y,cool_y,date,funny_y,stars_y,text,useful_y,user_id_y
0,---0hl58W-sjVTKi5LghGw,4,Y6qylbHq8QJmaCRSlKdIog,Ae4ABFarGMaI5lk1i98A0w,0,0,0,5adf69e796c8f7b6476e1feb,Ae4ABFarGMaI5lk1i98A0w,0,2016-07-24,0,4,Ate the momos during the momo crawl.. Was the ...,0,Y6qylbHq8QJmaCRSlKdIog
1,---OVMmOizYt30bx3-D0gA,5,WH0D3U5UAtNy0X5U5ZeJnw,ixGb4NLfgD90wZHlw94rLw,6,5,1,5adf6a3396c8f7b647a90915,ixGb4NLfgD90wZHlw94rLw,5,2015-08-12,1,5,Went in yesterday to place an order to take ho...,6,WH0D3U5UAtNy0X5U5ZeJnw
2,--0pfY3vQilgl20btE0fVQ,4,wbYWgTS-vzgfVYV_C1qhig,5BPv_vAnRp5EKdmacqvrLw,5,3,2,5adf6a4e96c8f7b647bd9be2,5BPv_vAnRp5EKdmacqvrLw,3,2013-05-19,2,4,An absolute favourite of mine....I don't under...,5,wbYWgTS-vzgfVYV_C1qhig
3,--0y7xOZPpiGD5d6vgfafg,1,EUdDBB1fHLq86T0aI9VLtg,vh1tPEaPioD78QmoqnWXpw,1,0,1,5adf69fb96c8f7b6477d408d,vh1tPEaPioD78QmoqnWXpw,0,2017-03-12,1,1,Very unimpressed by their pad Thai.. one of th...,1,EUdDBB1fHLq86T0aI9VLtg
4,--1sfdWmiYGbTxpGTy-3ZQ,1,hZgREoZPC30EEilayde2Ug,qAg8GkN9rforFWmGxBqVqA,2,0,0,5adf6a0796c8f7b64787061d,qAg8GkN9rforFWmGxBqVqA,0,2016-06-20,0,1,Arrived at 1:40pm on a Sunday just to get flat...,2,hZgREoZPC30EEilayde2Ug
5,--1Uj2FGirXqBmi_oA6-qg,5,-BJIu6TWs26q2yxozPAbEg,yxdmX8dXk5Se9zJ4WAW1pw,3,0,0,5adf6a0996c8f7b64788b465,yxdmX8dXk5Se9zJ4WAW1pw,0,2014-07-13,0,5,Outstanding food and service. \n\nThe atmosphe...,3,-BJIu6TWs26q2yxozPAbEg
6,--1U_KyCEPuYg2a1q03FbQ,5,0tpJmUYvbQSRbNa6DI0WVA,-yZXurhfOYlNKmH0ZyNGIQ,0,0,0,5adf6a2696c8f7b6479ecea4,-yZXurhfOYlNKmH0ZyNGIQ,0,2016-11-22,0,5,The one thing that most pleasantly surprised m...,0,0tpJmUYvbQSRbNa6DI0WVA
7,--242HMKec9b90td9y_s5g,5,3fR4DeE_eSa7SbzuC_fsaQ,i1Xkm22_96GNVVOXSUoL0A,0,0,0,5adf6a1396c8f7b6478fedfc,i1Xkm22_96GNVVOXSUoL0A,0,2016-02-12,0,5,"We went to Captain's Boil yesterday, and could...",0,3fR4DeE_eSa7SbzuC_fsaQ
8,--2WluEwS6KHmD-2v2V5sw,5,FykmLnIuh6oPaSPJ3RLB_A,aLcFhMe6DDJ430zelCpd2A,0,0,1,5adf6a4b96c8f7b647bb5f16,aLcFhMe6DDJ430zelCpd2A,0,2014-01-04,1,5,Absolutely amazing!! We just ordered take out ...,0,FykmLnIuh6oPaSPJ3RLB_A
9,--3sDRa8FTI_cKLF4eUAeQ,2,dhPp4t9_fD0R0eNsp_gYZw,d6G0w_e3GaA90NSQeovShA,0,0,0,5adf6a2e96c8f7b647a50afd,d6G0w_e3GaA90NSQeovShA,0,2015-11-08,0,2,Can't say that this is the best Pho place in t...,0,dhPp4t9_fD0R0eNsp_gYZw


In [149]:
tf = TfidfVectorizer(analyzer='word', min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(dfReviews['text'])

In [151]:
import pickle
f = open("./tfidf_matrix.pickle","wb")
pickle.dump(tfidf_matrix,f)
f.close()
#tfidf_matrix.to_pickle('tfidf_matrix.pickle')

In [23]:
f = open("./tfidf_matrix.pickle","rb")
tfidf_matrix = pickle.load(f)
f.close()


In [19]:
dfReviews.to_pickle('./reviews_with_text.pickle')

In [24]:
dfReviews = pd.read_pickle('./reviews_with_text.pickle')

In [20]:
dfReviews.loc[dfReviews['user_id_x'] == 'FykmLnIuh6oPaSPJ3RLB_A'].sort_values('stars_x', ascending=False).head(10).index

Int64Index([8, 82714, 198876, 182015, 158315, 130323, 120560, 75820, 63653,
            44702],
           dtype='int64')

In [29]:
cosine_similarities = cosine_similarity(tfidf_matrix[8], tfidf_matrix)


In [49]:
cosine_similarities[0].argsort()

array([111963, 318869, 121224, ..., 254312, 211677,      8])

In [28]:
[i for i in cosine_similarities[0].argsort()[:-10:-1]]

[8, 211677, 254312, 282762, 307740, 209843, 163535, 356009, 344614]

In [132]:
def item(id):
    return dfReviews.loc[dfReviews['review_id'] == id]

In [146]:
def get_recommendation_by_review(review_id):
    index = item(review_id).index[0]
    cosine_similarities = cosine_similarity(tfidf_matrix[index], tfidf_matrix)
    for i in cosine_similarities[0].argsort()[:-10:-1]:
        print(list(db.business.find({'business_id': dfReviews.loc[i]['business_id_x']}))[0]['name'])

In [148]:
get_recommendation_by_review('---0hl58W-sjVTKi5LghGw')

Himalayan Kitchen
Himalayan Kitchen
Loga's Corner
Tibet Kitchen
Everest Cafe & Bar
Shangrila Tibetan & Asian Cuisine
Loga's Corner
Kathmandu Restaurant
Loga's Corner


In [64]:
#obtener los 10 reviews mas favorables del usuario
review_indexes = dfReviews.loc[dfReviews['user_id_x'] == 'zsZVg16yjZu5NIiS0ayjrQ'].sort_values('stars_x', ascending=False).head(10).index

businesess_count = {}
businesses_top = {}
for index in review_indexes:
    # Obtener los revioew más similares a cada uno de esos 10 reviews del usuario
    cosine_similarities = cosine_similarity(tfidf_matrix[index], tfidf_matrix)
    # Los 10 reviews más similares al review
    for i in cosine_similarities[0].argsort()[:-10:-1]:
        business_id = dfReviews.loc[i]['business_id_x']
        if businesess_count.get(business_id):
            # Contador de cuantas veces aparece un negocio recomendado en total
            businesess_count[business_id] = businesess_count[business_id] + 1
            # Valor máximo del negocio recomendado
            businesses_top[business_id] = max(cosine_similarities[0][i], businesses_top[business_id])
        else:
            businesess_count[business_id] = 1
            businesses_top[business_id] = cosine_similarities[0][i]
        
top_businesses_count = {}
max = 10
i = 0
# Los 10 negocios que más aparecen sin contar los que fueron del review.
for business in sorted(businesess_count, key=businesess_count.get, reverse=True):
    if i >= max:
        break
    if businesses_top[business] < 1:
        top_businesses_count[business] = businesses_top[business]
        i = i + 1

['OBPiT9OyFDaTfaLRwfqlSg', 'Chv9fT9lXH9sD8tNME3U7w', 'QaNfzjAecuJXz1Je8UQhEA', 'ofw8aDSEg1HoQdmCgvLtaQ', 'CK6ZXaCAX7fc96tSWMp2hw', '1NXNqXI1xYtnodI7pVe9ig', 'nUVnnr5dYTXLLFwiCAp22Q', 'iS44aNB5h7bkAV3dzm9Fgw', 'Qesgux3MDYDYaqlsGUXqMQ', 'EL-iUP2pr6aJE2ZRVyNwyA']
['Qesgux3MDYDYaqlsGUXqMQ', 'Chv9fT9lXH9sD8tNME3U7w', 'EL-iUP2pr6aJE2ZRVyNwyA', 'UDk7qUVIetK3Y3NuARguqg', 'Iadni8K9IgCJe2yB3fpgTw', 'rxA9c0_XObabVL1WCTA4FA', '7ODXq--HE7QpzvWwgk5rMA', 'kiGc-yk1z1JgqTq7t6CcwA', 'irWRLV5HmmMCrwc56OM2SA', 'gbjIwB_r6y4YhaZMwEK6mw']
{'CK6ZXaCAX7fc96tSWMp2hw': 1.0000000000000004, 'rxA9c0_XObabVL1WCTA4FA': 0.21674710650833606, '7ODXq--HE7QpzvWwgk5rMA': 0.20678856596504347, 'kiGc-yk1z1JgqTq7t6CcwA': 0.17674288616412617, 'irWRLV5HmmMCrwc56OM2SA': 0.16455622343928092, 'gbjIwB_r6y4YhaZMwEK6mw': 0.1633991818591224, 'bRntmjztJ4dFDA_LYiEECw': 0.15835886630076804, 'ofw8aDSEg1HoQdmCgvLtaQ': 1.0000000000000002, 'NUXDfdxb5SlGSw3Y3Yidpw': 0.2559631066427752, '8PZGCtgG5MZKvz9i3B9gCA': 0.2243622785935137, '7IOt1Sahnsy

In [65]:

for business in sorted(top_businesses_count, key=top_businesses_count.get, reverse=True):
    print(list(db.business.find({'business_id': business}))[0]['name'])

Bloor & Kennedy Flower Shop
The Sewing Studio
Baker and Scone
Banjara Indian Cuisine
Banjara Indian Cuisine
Sneaky Dee's
Barberian's Steak House
Zuki Japanese Restaurant
K & K Specialty
Brick Street Bakery
